## 5.7 Hands On: Neural Networks in Python
There are many tool kits that support implementation of neural networks,
often based on Python, currently the most popular scripting language.

In the following chapters, I will go over how to implement neural
machine translation models. You can type the following commands in
the interactive Python interpreter and inspect what they compute.

### 5.7.1 Data Structures and Functions in Numpy
Let us start with the inference and training I described in this chapter.
For now, we do not use any dedicated neural network tool kit. However,
we do the advanced math library numpy, which supports computation
with vectors, matrices, and other tensors

In [1]:
import math
import numpy as np

In computer science, the typical data structure to represent vectors,
matrices, and higher-order tensors is an array. Numpy has its own array
data structures for which it defines basic tensor operations such as
addition and multiplication.

Here is how we represent the parameters for our example feed-forward neural network, which computes xor, i.e., the weight matrices
W and W<sub>2</sub> and the bias vectors b and b<sub>2</sub>.

In [2]:
W = np.array([[3, 4], [2, 3]])
b = np.array([-2, -4])
W2 = np.array([5, -5])
b2 = np.array([-2])


The sigmoid that we use as activation function is not already
provided by Numpy, so we need to define it ourselves. The function
operates element-wise on vectors, so we need to signal this to numpy
with @np.vectorize. We define the sigmoid(x) function and it's derivative sigmoid'(x)

> $sigmoid(x) = \frac{1}{1+e^{-x}}$

In [3]:
@np.vectorize
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

@np.vectorize
def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

The input and output pair from our example was (1,0) → 1. So, we
need to define these as well as vectors, thus as numpy arrays.


In [4]:
x = np.array([1, 0])
t = np.array([1])

### 5.7.2 Forward Computation
Now, we have all the pieces in place to carry out inference in our neural
network. Using matrix and vector representations the computation from
the input to the hidden layer is

> $sigmoid(x) = \frac{1}{1+e^{-x}}$

> $s = Wx + b$

> $h = sigmoid(s)$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(5.35)


The computation from the hidden layer to the output layer is:

> $z = W_{2}h +b$

> $y = sigmoid(z)$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(5.36)

Numpy makes it very easy to translate this into Python code. For the
multiplication of a weight matrix with a vector, we need the dot product.
Note that the default multiplication (*) is performed element-wise.

In [5]:
s = W.dot(x) + b
h = sigmoid(s)

z = W2.dot(h) + b2
y = sigmoid(z)

You can check the value of the computed output

In [6]:
y
# expected: array([0.7425526])

array([0.7425526])

### 5.7.3 Backward Computation
The next step is training via back-propagation. Recall that we start with
the computed error and compute gradients of the weights with respect to
the error. These gradients are then scaled with a learning rate and used
to update parameters.

So, first we need to compute the error between the computed output
y and the target value t. We used the L2 norm for this, i.e., $E = \frac{1}2{(t - y)^{2}}$.

In [9]:
error = 1/2 * (t - y)**2

We also need to set the learning rate μ. This is typically a small
number like 0.001 but here we just use 1


In [8]:
mu = 1

We went over quite a bit of math to derive formulas for the gradients
$\frac{\partial E}{\partial W},
\frac{\partial E}{\partial b},
\frac{\partial E}{\partial W_{2}}, \text { and } \frac{\partial E}{\partial b_{2}}$.
Our update formulas simplified this a bit by first
computing an error terms δ2 and δ1 and then using them for the weight
updates.

For the updates of parameters between the hidden layer and the final
layer, our formulas first compute δ<sub>2</sub>, and then the weight updates for W<sub>2</sub>
and b<sub>2</sub> are

> \begin{equation}
\begin{aligned}
\delta_{2} &=(t-y) \operatorname{sigmoid}^{\prime}(z) \\
\Delta W_{2} &=\mu \delta_{2} h \\
\Delta b_{2} &=\mu \delta_{2}
\end{aligned}
\end{equation}

Each equation can be formulated as one line in Python.

In [10]:
delta_2 = ( t - y ) * sigmoid_derivative( z )
delta_W2 = mu * delta_2 * h
delta_b2 = mu * delta_2

The update formulas for the parameters connecting the input layer
to the hidden layer are quite similar, partly due to our introduction of the
concept of an error term δ. The value for the this error term δ1 for first
layer is computed partly based on the value for the error term δ2 for the
second layer. This is back-propagation of the error in action:

>$\begin{equation}
\begin{aligned}
\delta_{1} &=W \; \delta_{2} \cdot \operatorname{sigmoid}^{\prime}(s) \\
\Delta W &=\mu \; \delta_{1} \; h \\
\Delta b &=\mu \; \delta_{1}
\end{aligned}
\end{equation}$

Also, the Python code is quite similar.

In [11]:
delta_1 = delta_2 * W2 * sigmoid_derivative( s )
delta_W = mu * np.array([ delta_1 ]).T * x
delta_b = mu * delta_1

### 5.7.4 Repeated Use of the Chain Rule
Let us take another view at the backward pass, which will also serve
as a segue to the next chapter. You may have already forgotten how we
derived at the formulas for the weight updates, and your head may be
hurting from the flashbacks to high school math.

However, it is actually simpler than you may believe. To derive
weight updates through back-propagation, we rely heavily on the chain
rule. Viewed from a high level, we have a computation chain, which is
a sequence of function applications, such as matrix multiplication and
activation functions.

Consider the part of the calculation that connects the target output
vector t, hidden vector h, the weight matrix W2, and the bias term b2 to
the error E:

>$\begin{equation}
E=\mathrm{L} 2\left(\operatorname{sigmoid}\left(W_{2} h+b_{2}\right), t\right)
\end{equation}$

Consider the computation chain from the parameter matrix W<sub>2</sub> to
E. We first have a matrix multiplication, then a vector addition, then the
sigmoid and finally the L2 norm. To compute \begin{equation}
\frac{\partial E}{\partial W_{2}}
\end{equation}, we treat the other values $(t, b2, h)$ as constants.
Abstractly, we have a computation of the
form $y = f(g(h(i(x))))$, with the weight matrix W<sub>2</sub> as input value x and
the error $E$ as output value $y$.

To obtain the derivative, our go-to rule here is the chain rule. Let us
play this through for the simpler example of just two chained functions
$f(g(x))$:

>\begin{equation}
\begin{aligned}
F(x) &=f(g(x)) \\
F^{\prime}(x) &=f^{\prime}(g(x)) g^{\prime}(x)
\end{aligned}
\end{equation}

Using different notation to clarify what we take the derivative over:

>\begin{equation}
\frac{\partial}{\partial x} f(g(x))=\frac{\partial}{\partial g(x)} f(g(x)) \frac{\partial}{\partial x} g(x) .
\end{equation}

Let us be explicit about the intermediate variable g(x) by naming it $a$:

>\begin{equation}
\begin{aligned}
a &=g(x) \\
\frac{\partial}{\partial x} f(g(x)) &=\frac{\partial}{\partial a} f(a) \frac{\partial}{\partial x} a
\end{aligned}
\end{equation}

So, we need to compute the derivatives of the elementary functions
$f$ and $g$ with respect to their inputs. We also need the values computed in
the forward computation since we will plug them into these derivatives.
Let us now work through the computation chain, starting from the
end. The last computation is the computation of the error:

>\begin{equation}
\begin{aligned}
E(y) &=\frac{1}{2}(t-y)^{2} \\
\frac{\partial}{\partial y} E(y) &=t-y .
\end{aligned}
\end{equation}

Let us do this in Python.

In [12]:
d_error_d_y = t - y

Continue on to the next step backward, the sigmoid activation function.
It processed the intermediate value $z$. Since we are trying to compute
gradients backward, we are still interested in the derivative of the error
with respect to this value. So, this is where we now deploy the chain
rule. Refer to Equation 5.42 with the substitutions $y → a$, $z → x$,
$sigmoid → g$, and $E → f$ in mind:

>\begin{equation}
\begin{aligned}
y &=\operatorname{sigmoid}(z) \\
\frac{\partial}{\partial z} y &=\operatorname{sigmoid}^{\prime}(z) \\
\frac{\partial}{\partial z} E(\operatorname{sigmoid}(z)) &=\frac{\partial}{\partial y} E(y) \frac{\partial}{\partial z} y
\end{aligned}
\end{equation}

We already computed \begin{equation}\frac{\partial}{\partial y} E(y)\end{equation} in the previous step. So, we reuse that
and multiply it with the derivative of the sigmoid, applied to the input
value $z$ at this step.

This is how it looks in Python.

In [13]:
d_y_d_z = sigmoid_derivative( z )
d_error_d_z = d_error_d_y * d_y_d_z

Let us take a closer look at one more step to get a good feeling for how
the gradient computation progresses backward through the computation
chain. The computation that computes $z$ is \begin{equation}W_{2} h+b_{2}\end{equation}. Here we are
interested in several gradients. One each for the parameters $W_{2}$ and $b_{2}$
to apply weight updates and one for the hidden layer values h to proceed
with the backward computation.

Let us look at just the derivatives for $W_{2}$:

>\begin{equation}
\begin{aligned}
z &=W_{2} h+b_{2} \\
\frac{\partial}{\partial W_{2}} z &=h \\
\frac{\partial}{\partial W_{2}} E(\operatorname{sigmoid}(z)) &=\frac{\partial}{\partial z} E(\operatorname{sigmoid}(z)) \frac{\partial}{\partial W_{2}} z
\end{aligned}
\end{equation}

Again, we already computed \begin{equation}\frac{\partial}{\partial z} E(\operatorname{sigmoid}(z))\end{equation} in the previous step, so
we can reuse it here.

In [14]:
d_z_d_W2 = h
d_error_d_W2 = d_error_d_z * d_z_d_W2

The gradient <code>d_error_d_W2</code> that we computed at this point
matches <code>delta_W2</code> (see the previous section) since the learning rate $μ$
is 1. We can check this by comparing the computed values by our code.

In [15]:
d_error_d_W2
# expected: array([0.03597961, 0.00586666])
delta_W2
# expected: array([0.03597961, 0.00586666])

array([0.03597961, 0.00586666])

The computations for the other gradients for the forward step
$z = W_{2}h + b_{2}$ follow the same logic.

In [ ]:
d_z_d_b2 = 1
d_error_d_b2 = d_error_d_z * d_z_d_b2
d_z_d_h = W2
d_error_d_h = d_error_d_z * d_z_d_h

The computations for the layer connecting the input x to the hidden
values $h$ matches closely with what we just presented in detail.

In [ ]:
d_s_d_h = sigmoid_derivative( s )
d_error_d_s = d_error_d_h * d_s_d_h
d_W_d_s = x
d_error_d_W = np.array([ d_error_d_s ]).T * d_W_d_s
d_b_d_s = 1
d_error_d_b = d_error_d_s * d_b_d_s